## <font color=009DD2> <span style="background-color: #A9C4C0"> Imports

In [116]:
import sys,json
import glob, os
sys.path.append('/media/miplab-nas2/Data3/BMPD/hc_project/analysis/code/')

from seed_to_voxels_inprogress_nk import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=009DD2> <span style="background-color: #A9C4C0"> Initialization

### <font color=009DD2> I Configuration & parameters </font>

In [120]:
with open('../config/config_NK.json') as config_file:
    config = json.load(config_file) # load config file

# Parameters (maybe we should move them to config?)
signal='ai'
seed_names=['icap1','icap2','icap3','icap4','icap5','icap6','icap7','icap8','icap9'] # Define seed names (here, icap numbers)
target_name=['brain'] # Define structure to use as target (brain or spinal cord)
ktot = 9 # Total number of components to use (just to define path)


### <font color=009DD2> II Select files </font>

In [121]:
data_target=[];ts_seed_txt=[];ts_target_txt=[]
for seed_name in seed_names:
    for subject_name in config['list_subjects']:
        data_target.append(config['main_dir'] + '1_nilearn-based/3_func_smoothed/sub-'+ subject_name + '/TA_results/MNI/TotalActivation/Activity_inducing.nii')
        ts_seed_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) +'/'+seed_name+'/timeseries/sub-' + subject_name + '_' + seed_name + '_timeseries_ai')
        ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/sub-' + subject_name + '_' + target_name[0] + '_all_timeseries_ai')
        
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) + '/' +seed_name+'/'+target_name[0]+'_fc_maps/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) +'/'+seed_name+'/'+target_name[0]+'_fc_maps/')

mask_target=config['main_dir'] + config['masks'][target_name[0].split('_')[0]]
 
# Create directories if needed
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/'):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all')
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/') # folder to store timeseries extraction


### <font color=009DD2> III Initialize class based on this </font>

In [123]:
seed2voxels=Seed2voxels(config,signal,seed_names,target_name) # initialize the function

## <font color=009DD2> <span style="background-color: #A9C4C0"> Data extraction


### <font color=009DD2> I. Time series extraction - Seed </font>

In [124]:
seed_timeseries=seed2voxels.extract_data(timeseries_txt=ts_seed_txt,
                                         run="load",n_jobs=8) # Load iCAP timecourses as seed

### <font color=009DD2> II. Time series extraction - Target </font>

In [125]:
target_timeseries=seed2voxels.extract_data(img=data_target, mask=mask_target, timeseries_txt=ts_target_txt,
                run="load",n_jobs=8) # Load voxelwise brain AI as target

## <font color=009DD2> <span style="background-color: #A9C4C0"> Correlation analysis

In [126]:
output_img={};
for seed_name in seed_names:
    output_img[seed_name]=[]
    output_img[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) + '/'+seed_name+'/'+target_name[0]+'_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_' + seed_name
    seed2voxels.correlation_maps(seed_timeseries,target_timeseries,mask=mask_target,output_img=output_img[seed_name],Fisher=True,n_jobs=8)


/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/kinany/ana

## <font color=009DD2> <span style="background-color: #A9C4C0"> Smoothing

In [128]:
from nilearn import image
img_smoothed=image.smooth_img(output_img[seed_name]+'_zcorr.nii',[6,6,6])
img_smoothed.to_filename(output_img[seed_name].split('.')[0] + '_s.nii' )